# Perform Surface Network Mapping 'Overlap' Experiment

### Authors: William Drew, Alexander Cohen, Christopher Lin

Notes:
- This notebook requires the NIMLAB Python 3 environment as a kernel. Directions at: (https://github.com/nimlab/software_env)
- This notebook is a combination of the old Simple Analysis Notebook and the overlap_maps notebooks (does the same thing).

In [ ]:
## Packages and environmental settings:

##Packages:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from tqdm import tqdm
from matplotlib import pyplot as plt
from nilearn import image, plotting, regions, datasets
from nimlab import datasets as nimds
from nimlab import functions as nimf
from nimlab import surface as nimsurf
fsaverage = datasets.fetch_surf_fsaverage("fsaverage5")
%matplotlib inline

# Input/Output Setup

### 1. Provide your email address for future reference:

In [ ]:
# Enter your email address here in quotes:
creator_email = "wdrew@bwh.harvard.edu"

### 2. Where are your data? and where do you want the results to go?
NOTE: Provide a csv file created from the `Preprocessing` or `xnat_grabber` notebooks that points to your Lesions and fcMaps (will also work with older csv files from `xnat_gate`).

In [ ]:
# Specify the path to your dataset.csv file

dataset_csv = ""
output_dir = ""

# EXAMPLE dataset_csv = "Prosopagnosia_w_Yeo1000.csv"
# EXAMPLE output_dir = "2_LNM_of_Prosopagnosia_w_Yeo1000"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

img_csv = pd.read_csv(dataset_csv)
print("I found", len(img_csv), "participants:")
img_csv.head()

# Before we look at the *lesion network* overlap, let's review the *lesion* overlap map

In [ ]:
# Generate an "N-image" and a "Coverage Map", then show me:
lh_lesions = [nib.load(entry).agg_data() for entry in img_csv['roi_fs5_L']]
rh_lesions = [nib.load(entry).agg_data() for entry in img_csv['roi_fs5_R']]

lh_lesion_overlap = nimsurf.new_gifti_image(np.mean(lh_lesions, axis=0) * len(img_csv))
rh_lesion_overlap = nimsurf.new_gifti_image(np.mean(rh_lesions, axis=0) * len(img_csv))

lh_lesion_mask = nimsurf.new_gifti_image(lh_lesion_overlap.agg_data() >= 1)
rh_lesion_mask = nimsurf.new_gifti_image(rh_lesion_overlap.agg_data() >= 1)

fig, ax = plt.subplots(4,2,subplot_kw=dict(projection='3d'),figsize=(8,12))
fig.tight_layout()
plotting.plot_surf_stat_map(fsaverage.pial_left, lh_lesion_overlap.agg_data(), hemi="left", view="lateral", axes=ax[0,0],cmap="Spectral_r",title='ROI Overlap');
plotting.plot_surf_stat_map(fsaverage.pial_right, rh_lesion_overlap.agg_data(), hemi="right", view="lateral", axes=ax[0,1],cmap="Spectral_r");
plotting.plot_surf_stat_map(fsaverage.pial_left, lh_lesion_overlap.agg_data(), hemi="left", view="medial", axes=ax[1,0],cmap="Spectral_r");
plotting.plot_surf_stat_map(fsaverage.pial_right, rh_lesion_overlap.agg_data(), hemi="right", view="medial", axes=ax[1,1],cmap="Spectral_r");
plotting.plot_surf_stat_map(fsaverage.pial_left, lh_lesion_mask.agg_data(), hemi="left", view="lateral", axes=ax[2,0],cmap="bwr",title='ROI Coverage');
plotting.plot_surf_stat_map(fsaverage.pial_right, rh_lesion_mask.agg_data(), hemi="right", view="lateral", axes=ax[2,1],cmap="bwr");
plotting.plot_surf_stat_map(fsaverage.pial_left, lh_lesion_mask.agg_data(), hemi="left", view="medial", axes=ax[3,0],cmap="bwr");
plotting.plot_surf_stat_map(fsaverage.pial_right, rh_lesion_mask.agg_data(), hemi="right", view="medial", axes=ax[3,1],cmap="bwr");

# Lesion Network Mapping

### 1. Load the fcMap Tmaps and generate a plot relating T-thresholds to the maximum percentage of overlap
#### Select a brain mask for visualization
- Default brain mask is `MNI152_T1_2mm_brain_mask_dil`
- A more strict brain mask option available is `MNI152_T1_2mm_brain_mask`
- Additional options are listed in `nimfs.check_mask()` or you can specify a path to a binary mask of your choosing
    
NOTES:
- This can be useful for selecting an appropriate threshold.
- See [Cohen and Fox, 2020](https://doi.org/10.1093/brain/awaa095) for details.


In [ ]:
# Specify the range of T score levels to test as (start, end+1, increment).

t_levels = range(4,16,1)
# EXAMPLE t_levels = range(4,16,1)

# You can change this mask to a more restrictive one for visualization purposes
# Can be a name of a mask in nimlab.datasets or a path to a binary mask nifti file.
lh_mask="fs5_mask_lh"
rh_mask="fs5_mask_rh"

lh_t_maps = list(img_csv['surfLhT'])
rh_t_maps = list(img_csv['surfRhT'])

max_positives = []
max_negatives = []
for t in tqdm(t_levels):
    lh_pos_overlap, lh_neg_overlap = nimsurf.get_pos_neg_overlap_maps(lh_t_maps, t, mask = lh_mask) 
    rh_pos_overlap, rh_neg_overlap = nimsurf.get_pos_neg_overlap_maps(rh_t_maps, t, mask = rh_mask) 
    max_positives.append(np.max([np.nanmax(lh_pos_overlap.agg_data()), np.nanmax(rh_pos_overlap.agg_data())]))
    max_negatives.append(np.max([np.nanmax(lh_neg_overlap.agg_data()), np.nanmax(rh_neg_overlap.agg_data())]))


plt.plot(t_levels, max_positives, label='Positive')
plt.plot(t_levels, max_negatives, label='Negative')
plt.title("T-score levels vs. peak LNM overlap")
plt.xlabel("T-score threshold")
plt.ylabel("Max overlap")
plt.legend();

### 2. At a specific threshold, show me the spatial overlap at various percentages

NOTES:
- We have typically considered a rough guideline of $\geq$90-95% overlap to be important (100% is easiest to interpret of course)
- Be mindful of how you think about thresholds for the overlap:
  - Here, I have modified the code to show voxels **$\geq$ instead of $\gt$thresholds**, so if a voxel is present in exactly 75% of subjects, I am displaying it here.
  - FSLeyes (and nilearn) default to show data $\gt$threshold, so if you load the `LNM_fraction_xxx.nii.gz` and set the range to 0.75 to 1, you are viewing voxels **$\gt$0.75, not $\geq$0.75**.
  - `direction` defaults to `"twosided"`, which considers both positive and negative connectivity when calculating overlaps. You can also set `direction` to `"positive"` or `"negative"` to consider only positive or negative connectivity when calculating overlaps, respectively.
  - `mode` defaults to `"fraction"`, which plots the overlap maps as a fraction of the total number of subjects. You can also set `mode` to `"count"` to plot the overlap maps as a count of the number of subjects that overlap at any particular region. 


In [ ]:
# Specify a T-score threshold

T_level = 9
# EXAMPLE T_level = 9

# Specify a range of overlap fractions to examine
range_to_examine = np.linspace(0.75,1,6)

nimsurf.plot_overlap_surface_map_to_screen(
    lh_t_maps, rh_t_maps, T_level, range_to_examine, lh_mask = lh_mask, rh_mask = rh_mask,
    direction = "twosided",
    mode = "fraction"
)

### 3. Final required step: Write your overlap maps to files

NOTE: I put this in a separate cell, so you can re-run the prior cell multiple times to view the differences. Once you've selected the T-score thresholds you want to use, specify them below to write the overlap maps to files
- `direction` defaults to `"twosided"`, which considers both positive and negative connectivity when calculating overlaps. You can also set `direction` to `"positive"` or `"negative"` to consider only positive or negative connectivity when calculating overlaps, respectively.

In [ ]:
# Specify the specific T-score thresholds you want to print in the array here:
for t in [5, 7, 9]:
    print("Writing Files for a T>",str(t))
    nimsurf.write_overlap_map_to_file(
        lh_t_maps,
        rh_t_maps,
        t,
        output_dir,
        lh_mask="fs5_mask_lh",
        rh_mask="fs5_mask_rh",
        direction="twosided",
    )